<div>
<img src="figures/svtLogo.png"/>
</div>
<h1><center>Mathematical Optimization for Engineers</center></h1>
<h2><center>Lab 14 - Uncertainty</center></h2>

We want to optimize the total annualized cost of a heating and electric power system. Three different technologies are present: 
- a gas boiler
- a combined heat and power plant
- a photovoltaic module

We first the the nominal case without uncertanties. 
Next, we will consider a two-stage approach to consider uncertainties in the electricity demand and the power producable via PV. 
Uncertain variables are the solar power and the power demand. 

In [89]:
# import cell
from scipy.optimize import minimize, NonlinearConstraint, Bounds
import numpy as np

In [90]:
class Boiler():
    """Boiler 
    Gas in, heat out
    """
    
    def __init__(self):
        self.M = 0.75  
        
    def invest_cost(self, Qdot_nom):
        inv = 100 * Qdot_nom ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas = 60
        cost_gas_oper = Qdot_nom * cost_gas * op_load
        
        return cost_gas_oper
    
    def heat(self, Qdot_nom, op_load):
        eta_th = 0.9 - (1 - op_load) * 0.05
        return Qdot_nom * op_load * eta_th
    

In [91]:
class CHP():
    """Combined-heat-and-power (CHP) engine 
    Gas in, heat and power out
    """

    def __init__(self):
        self.c_ref = 150
        self.M = 0.80  # [-], cost exponent
        self.cost_gas = 60
    
    def invest_cost(self, Qdot_nom):
        inv = self.c_ref * (Qdot_nom) ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas_oper = Qdot_nom * op_load * self.cost_gas
        return cost_gas_oper
    
    def elec_out(self, Qdot_nom, op_load):
        eta_el = 0.3 - (1 - op_load) * 0.1
        out_pow = eta_el * Qdot_nom * op_load
        return out_pow
    
    def heat(self, Qdot_nom, op_load): 
        eta_th = 0.6 - (1-op_load) * 0.05  
        return Qdot_nom * eta_th * op_load


In [92]:
class PV:
    """Photovoltaic modules (PV) 
    solar 
    """ 
    
    def __init__(self): 
        self.M = 0.9  # [-], cost exponent
       
    def invest_cost(self, p_nom):
        inv = 200 * p_nom ** self.M
        return inv
    
    def oper_cost(self, out_nom): 
        return 0
    
    def elec_out(self, p_nom, op_load, solar):
        return p_nom * op_load * solar
    

In [93]:
def objective_function(x, PV, Boiler, CHP, scenarios):
    total_cost = 0
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 
    
    # investment costs
    investment_costs = (PV.invest_cost(design_PV) + 
                       Boiler.invest_cost(design_boiler) + 
                       CHP.invest_cost(design_CHP))
    
    # expected operating costs
    total_operating_costs = 0
    for i, scenario in enumerate(scenarios):
        PV_load = x[(i+1)*3]
        boiler_load = x[(i+1)*3+1]
        CHP_load = x[(i+1)*3+2]
        operating_costs = Boiler.oper_cost(design_boiler, boiler_load) + \
                          CHP.oper_cost(design_CHP, CHP_load)
        
        total_operating_costs += (operating_costs * scenario["p"])
    
    
    total_cost = investment_costs + total_operating_costs

    return total_cost

In [94]:
def constraint_function(x, PV, Boiler, CHP, scenarios): 
    heat_demand = 200
    
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 

    # loop over all uncertainties
    c = []
    
    for i, scenario in enumerate(scenarios):
        PV_load = x[(i+1)*3]
        boiler_load = x[(i+1)*3+1]
        CHP_load = x[(i+1)*3+2]
        
        # heat demand
        heat = CHP.heat(design_CHP, CHP_load) + \
               Boiler.heat(design_boiler, boiler_load)
        c.append(heat_demand - heat)
        
        # electricty demand 
        elec = CHP.elec_out(design_CHP, CHP_load) + \
               PV.elec_out(design_PV, PV_load, scenario["solar"])
        c.append(scenario["elec"] - elec)
        
    return c

In [95]:
def print_solution(x):
    print('PV design: ', x[0])
    print('Boiler design: ', x[1])
    print('CHP design: ', x[2])
    
    # print scenarios
    n_scenarios = int((len(x) - 3) / 3)
    for i_scenario in range(1, n_scenarios + 1): 
            print('Scenario ' + str(i_scenario) + ' PV load: ', x[3 * i_scenario])
            print('Scenario ' + str(i_scenario) + ' Boiler load: ', x[3 * i_scenario + 1])
            print('Scenario ' + str(i_scenario) + ' CHP load: ', x[3 * i_scenario + 2], end='\n\n')
    

In [96]:
# nominal case
scenario1 = {"p": 1.0, "solar":1.0, "elec": 100}
scenarios = [scenario1] # base scenario


In [97]:
# now consider different scenarios
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
# constraints need bounds
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200,200,200, 1,1,1 ]
# bounds for decision variables
lbs = [0, 0, 0, 0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1]
bnds = Bounds(lbs, ubs)


In [98]:
res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 25, 'iprint': 2, 'disp': True})


  NIT    FC           OBJFUN            GNORM
    1     8     4.230683E+04     1.697139E+04
    2    15     3.378399E+04     1.515439E+04
    3    22     3.433070E+04     1.215995E+04
    4    29     3.405367E+04     1.113483E+04
    5    36     3.171484E+04     1.148279E+04
    7    43     3.170808E+04     1.333695E+04
    9    50     3.170808E+04     1.405754E+04
Optimization terminated successfully    (Exit mode 0)
            Current function value: 31708.080336661325
            Iterations: 9
            Function evaluations: 50
            Gradient evaluations: 7


In [99]:
print_solution(res.x)

PV design:  99.99999977680706
Boiler design:  222.22222217945233
CHP design:  0.0
Scenario 1 PV load:  1.0
Scenario 1 Boiler load:  1.0
Scenario 1 CHP load:  1.0



In [100]:
# nominal 
# uncertanties: power demand and solar power (relative 1.0)
scenario1 = {"p": 0.4, "solar":1.0, "elec": 100}
scenario2 = {"p": 0.3, "solar":1.0, "elec": 120}
scenario3 = {"p": 0.3, "solar":0.5, "elec": 80}

# put scenarios together
scenarios = [scenario1, scenario2, scenario3]

In [101]:
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
# bounds and constraints
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
x_guess = [200,200,200] + [1, 1, 1] * len(scenarios)
lbs = [0, 0, 0] + [0, 0, 0] * len(scenarios)
ubs = [10000, 10000, 10000] + [1, 1, 1] * len(scenarios)
bnds = Bounds(lbs, ubs)


res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 15, 'iprint': 2, 'disp': True})

  NIT    FC           OBJFUN            GNORM
    1    14     4.368662E+04     9.896865E+03
    2    27     3.780995E+04     8.491301E+03
    3    40     3.822670E+04     7.547584E+03
    4    53     3.775750E+04     7.319111E+03
    5    66     3.775213E+04     6.631493E+03
    6    79     3.776393E+04     6.596803E+03
    7    92     3.776375E+04     6.599369E+03
    9   105     3.776385E+04     6.599323E+03
   10   118     3.776382E+04     6.599339E+03
   12   141     3.776385E+04     6.599339E+03
   13   164     3.776385E+04     6.599339E+03
   14   187     3.776365E+04     6.599339E+03
   15   209     3.776385E+04     6.599339E+03
Iteration limit reached    (Exit mode 9)
            Current function value: 37763.854880380386
            Iterations: 15
            Function evaluations: 209
            Gradient evaluations: 13


In [103]:
print_solution(res.x)

PV design:  79.99999999686838
Boiler design:  133.33333332272832
CHP design:  133.33333333948795
Scenario 1 PV load:  0.7499999871985024
Scenario 1 Boiler load:  0.9999999868880614
Scenario 1 CHP load:  0.9999999999991549

Scenario 2 PV load:  0.9999999904536151
Scenario 2 Boiler load:  0.9999999902223338
Scenario 2 CHP load:  0.9999999999993647

Scenario 3 PV load:  0.9999999885369261
Scenario 3 Boiler load:  0.999999994160683
Scenario 3 CHP load:  0.9999999999995447

